In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.ar_model import AR as AutoRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_chatelet = pd.read_csv("https://raw.githubusercontent.com/AlexanderOG/AI_CRI/main/qualite-de-lair-mesuree-dans-la-station-chatelet.csv", sep=";")
def date_formatting_other(timestamp):
    timestamp = str(timestamp)
    timestamp=timestamp[:19]
    return datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S")

def add_cols(df):
    df['hour'] = df['DATE/HEURE'].apply(lambda x: x.hour)
    df['weekday'] = df['DATE/HEURE'].apply(lambda x: x.weekday())
    df['month'] = df['DATE/HEURE'].apply(lambda x: x.month)

df_chatelet['DATE/HEURE'] = df_chatelet['DATE/HEURE'].apply(lambda x: date_formatting_other(x))
df_chatelet['NO'].replace({"<2": "2", "ND":"0", "nan":None}, inplace=True)
df_chatelet = df_chatelet.dropna()
col = ['NO', 'NO2', 'PM10', 'CO2', 'TEMP', 'HUMI']
for i in col:
  df_chatelet[i] = pd.to_numeric(df_chatelet[i], errors='coerce')
add_cols(df_chatelet)
df_chatelet.sort_values(by = 'DATE/HEURE', ascending=True, inplace = True)
df_chatelet['CO2'] = df_chatelet['CO2'].fillna(df_chatelet['CO2'].interpolate().fillna(method="backfill", axis=None))
df_chatelet['TEMP'] = df_chatelet['TEMP'].fillna(df_chatelet['TEMP'].interpolate().fillna(method="backfill", axis=None))
df_chatelet['HUMI'] = df_chatelet['HUMI'].fillna(df_chatelet['HUMI'].interpolate().fillna(method="backfill", axis=None))
df_chatelet['NO2'] = df_chatelet['NO2'].fillna(df_chatelet['NO2'].interpolate().fillna(method="backfill", axis=None))
df_chatelet['PM10'] = df_chatelet['PM10'].fillna(df_chatelet['PM10'].interpolate().fillna(method="backfill", axis=None))
df_chatelet.index = df_chatelet["DATE/HEURE"]
df_chatelet = df_chatelet.drop(['DATE/HEURE'], axis=1)
df_chatelet

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,NO,NO2,PM10,CO2,TEMP,HUMI,hour,weekday,month
DATE/HEURE,,,,,,,,,
2013-01-01 02:00:00,6,31.0,182.0,776.0,21.0,47.000000,2,1,1
2013-01-01 03:00:00,6,30.0,166.0,800.0,21.0,47.000000,3,1,1
2013-01-01 04:00:00,4,27.0,124.0,799.0,21.0,47.000000,4,1,1
2013-01-01 05:00:00,3,25.0,85.0,715.0,21.0,46.666667,5,1,1
2013-01-01 06:00:00,4,26.0,102.0,622.0,21.0,46.333333,6,1,1
...,...,...,...,...,...,...,...,...,...
2021-10-17 22:00:00,3,20.0,79.0,567.0,20.0,42.000000,22,6,10
2021-10-17 23:00:00,4,20.0,62.0,518.0,20.0,42.000000,23,6,10
2021-10-18 00:00:00,4,23.0,47.0,497.0,20.0,42.000000,0,0,10


In [ ]:
train = [:int(0.8*len(df_chatelet))]
test = [int(0.8*len(df_chatelet)):]
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit()

In [33]:
copy = df_chatelet.copy()

In [34]:
df_chatelet = copy[:24*30]
from sklearn.cluster import KMeans
n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(df_chatelet);

In [35]:
clusters = []
for i in range(n_clusters):
  cluster = df_chatelet.iloc[kmeans.labels_==i]
  clusters.append(cluster)

In [37]:
clusters[0].groupby("weekday")["weekday"].value_counts().sum()

53